In [2]:
import duckdb
import pandas as pd
from statsbombpy import sb

# DuckDB bağlantısı
con = duckdb.connect("bitirme.db")

# Tablo oluşturma (sadece bir kez çalışır)
con.sql("""
CREATE TABLE IF NOT EXISTS passes (
    id INTEGER,
    rowID INTEGER,
    x FLOAT,
    y FLOAT,
    pass_end_x FLOAT,
    pass_end_y FLOAT,
    pass_succes BOOLEAN,
    f3rd_pass BOOLEAN,
    pass_progression FLOAT,
    shot_assisted_pass BOOLEAN,
    goal_assisted_pass BOOLEAN,
    "cross" BOOLEAN,
    box_pass BOOLEAN,
    competitions_name TEXT,
    match_id INTEGER
)
""")

con.sql("""
CREATE TABLE IF NOT EXISTS fouls (
    id INTEGER,
    rowID INTEGER,
    type TEXT,
    player TEXT,
    related_events TEXT,
    foul_committed_advantage BOOLEAN,
    foul_committed_card TEXT,
    foul_committed_type TEXT,
    x FLOAT,
    y FLOAT,
    competitions_name TEXT,
    match_id INTEGER
)
""")

# Lig ve sezon bilgileri
competitions_and_seasons = [
    (11, 27, "2015/16 La Liga"),
    (9, 27, "2015/16 Bundesliga"),
    (2, 27, "2015/16 Premier League"),
    (12, 27, "2015/16 Serie A"),
    (7, 27, "2015/16 Ligue 1"),
    (43, 106, "FIFA World Cup 22"),
    (43, 3, "FIFA World Cup 18"),
    (55, 282, "Euro 2024"),
    (55, 43, "Euro 2020")
]

for competition, season, competitions_name in competitions_and_seasons:
    # Maçları al
    matches = sb.matches(competition_id=competition, season_id=season)
    events_extra_info = matches[["match_id", "season", "home_team", "away_team", "referee"]]

    for match_ID in matches["match_id"].unique():
        events_df = sb.events(match_id=match_ID)
        events_df = events_df.set_index("match_id").join(
            events_extra_info.set_index("match_id"), how="left", rsuffix='_match'
        ).reset_index()

        # Koordinatları ayır
        events_df[['x', 'y']] = events_df['location'].apply(pd.Series)
        events_df[['pass_end_x', 'pass_end_y']] = events_df['pass_end_location'].apply(pd.Series)

        # Paslar tablosunu oluştur
        passes = events_df[(events_df.type == "Pass")]
        passes = passes[passes["pass_outcome"] != "Injury Clearance"]
        shots = events_df[(events_df.type == "Shot")][["shot_key_pass_id", "shot_outcome", "shot_statsbomb_xg"]].dropna()

        passes_M = passes.set_index("id").join(
            shots.set_index("shot_key_pass_id"), how="left", lsuffix='_pass', rsuffix='_fromPass'
        )
        passes_M["pass_succes"] = passes_M.pass_outcome.isna()
        passes_M["f3rd_pass"] = (passes_M.x < 80) & (passes_M.pass_end_x > 80)
        passes_M["pass_progression"] = passes_M["x"] - passes_M["pass_end_x"]
        passes_M.loc[passes_M.pass_outcome.notna(), "pass_progression"] = None
        passes_M["shot_assisted_pass"] = passes_M.pass_outcome.isna() & (
            passes_M.pass_shot_assist.notna() | passes_M.pass_goal_assist.notna()
        )
        passes_M["goal_assisted_pass"] = passes_M.pass_outcome.isna() & passes_M.pass_goal_assist.notna()
        passes_M["cross"] = (passes_M.pass_cross == True)
        passes_M["box_pass"] = (
            (passes_M.x < 102) & ((passes_M.y < 18) | (passes_M.y > 62)) &
            (passes_M.pass_end_x > 102) & ((passes_M.pass_end_y > 18) | (passes_M.pass_end_y < 62))
        )
        passes_M["rowID"] = passes_M.index
        passes_M["competitions_name"] = competitions_name
        passes_M["match_id"] = match_ID

        # Fauller tablosunu oluştur
        foul_commited = events_df[(events_df.type == "Foul Committed")][
            ["id", "type", "player", "related_events", "foul_committed_advantage", "foul_committed_card", "foul_committed_type", "x", "y"]
        ].set_index("id")
        fouls = events_df[(events_df.type == "Foul Won")].explode("related_events")
        fouls_M = fouls.set_index("related_events").join(foul_commited, how="left", rsuffix='_foul_commitedJ')
        fouls_M = fouls_M[fouls_M["player_foul_commitedJ"].notna()]
        fouls_M["rowID"] = fouls_M.index
        fouls_M["competitions_name"] = competitions_name
        fouls_M["match_id"] = match_ID

        # Verileri DuckDB'ye ekle
        con.execute("INSERT INTO passes SELECT * FROM passes_M", {"passes_M": passes_M})
        con.execute("INSERT INTO fouls SELECT * FROM fouls_M", {"fouls_M": fouls_M})

# Bağlantıyı kapat
con.close()


/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


BinderException: Binder Error: table passes has 15 columns but 110 values were supplied